In [1]:
import os
import requests
from supabase import create_client, Client
from dotenv import load_dotenv
from datetime import datetime, timedelta

In [8]:
# Load environment variables
load_dotenv()

# Initialize clients
GROQ_API_KEY = os.getenv('GROQ_API_KEY')
GROQ_API_URL = "https://api.groq.com/openai/v1/chat/completions"

supabase_url = os.getenv('SUPABASE_URL')
supabase_key = os.getenv('SUPABASE_ANON_KEY')
supabase: Client = create_client(supabase_url, supabase_key)


🎵 Event Recommender Test 🎵

Fetching events from database...
Found 13 events

--- Test 1: 'I want to see some techno this weekend' ---
Recommendation:
"Hey! I've got some sick techno recs for you this weekend! 

1. SNEAK RAVE on Sep 2nd at XOYO - This one's a no-brainer! A rave sounds like the perfect spot for some techno vibes 

2. Steely Dancing on Sep 2nd at Ridley Road Market Bar - This might not scream techno, but it's a great chance to discover some new sounds. Plus, it's on a Saturday!

3. PARADOX on Sep 3rd at XOYO - Another XOYO event, another techno possibility! This one's on Sunday, so you've got a second chance to get your techno fix

Hope you find something that gets you moving!"

--------------------------------------------------

--- Test 2: 'Looking for house music events next week' ---
Recommendation:
"Hey! I've got some sick house music event recs for you in London next week! 

1. Final CUT - House, Hip Hop & Chart on 2025-09-03 at Egg London - This one's a no-brainer

In [3]:
def get_upcoming_events(days_ahead=7, limit=50):
    """
    Fetch upcoming events from Supabase
    """
    try:
        # Calculate date range
        start_date = datetime.now().strftime('%Y-%m-%d')
        end_date = (datetime.now() + timedelta(days=days_ahead)).strftime('%Y-%m-%d')

        # Query Supabase using correct column names
        result = supabase.table('resident_advisor')\
            .select('*')\
            .gte('Date', start_date)\
            .lte('Date', end_date)\
            .limit(limit)\
            .execute()

        return result.data
    except Exception as e:
        print(f"Error fetching events: {e}")
        return []


In [4]:
def format_events_for_llm(events):
    """
    Format events data for the LLM prompt
    """
    if not events:
        return "No events found for the specified dates."

    formatted_events = []
    for event in events[:30]:  # Limit to avoid token limits
        # Using correct column names: Artists, Venue, Event name, Date
        event_text = f"• {event.get('Event name', 'N/A')}"
        if event.get('Date'):
            event_text += f" - {event.get('Date')}"
        if event.get('Venue'):
            event_text += f" at {event.get('Venue')}"
        if event.get('Artists'):
            event_text += f" | Artists: {event.get('Artists')}"
        formatted_events.append(event_text)

    return "\n".join(formatted_events)

In [12]:
def generate_recommendations(user_request, events_data, context=None):
    """
    Use Groq to generate event recommendations with user context
    """
    # Build context information for the prompt
    context_info = ""

    if context:
        # Add user's music preferences
        if context.get('preferred_genres'):
            genres = ', '.join(context['preferred_genres'])
            context_info += f"\nUser's known music preferences: {genres}"

        # Add conversation history for continuity
        if context.get('conversation_history'):
            recent_history = context['conversation_history'][-4:]  # Last 2 exchanges
            context_info += f"\nRecent conversation context: {'; '.join(recent_history)}"

        # First-time user greeting
        if context.get('first_interaction', True):
            context_info += "\nThis is the user's first interaction - introduce yourself as their music buddy!"

    prompt = f"""You are a helpful event recommender and the user's virtual music buddy. A user has made this request: "{user_request}"
{context_info}

Here are the upcoming electronic music events in London:

{events_data}

Be conversational and laid-back. You can:
- Just chat about what they're looking for
- Suggest 1-2 events max (not a whole list!)
- Ask what they're in the mood for
- Sometimes just be supportive without pushing events
- Be attentive to what they actually want
- Use occasional animal emojis naturally 🐨
- Keep it short and friendly
- The context is a whatsapp chat

Don't be salesy or overwhelming. Match their energy."""

    try:
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {GROQ_API_KEY}"
        }

        data = {
            "model": "meta-llama/llama-4-scout-17b-16e-instruct",
            "messages": [
                {"role": "system", "content": "You are a knowledgeable electronic music event recommender and virtual music buddy who remembers user preferences."},
                {"role": "user", "content": prompt}
            ],
            "max_tokens": 500,
            "temperature": 0.7
        }

        response = requests.post(GROQ_API_URL, headers=headers, json=data)
        response.raise_for_status()

        return response.json()["choices"][0]["message"]["content"]
    except Exception as e:
        return f"Error generating recommendations 🐨: {e}"

In [6]:
def main():
    """
    Main function to test the recommender
    """
    print("🎵 Event Recommender Test 🎵\n")

    # Test user requests
    test_requests = [
        "I want to see some techno this weekend",
  #      "Looking for house music events next week",
 #       "What's on tonight?",
#        "I like melodic electronic music"
    ]

    # Fetch events
    print("Fetching events from database...")
    events = get_upcoming_events()
    print(f"Found {len(events)} events\n")

    if not events:
        print("No events found. Check your database connection and data.")
        return

    # Format events for LLM
    events_text = format_events_for_llm(events)

    # Test each request
    for i, request in enumerate(test_requests, 1):
        print(f"--- Test {i}: '{request}' ---")

        recommendation = generate_recommendations(request, events_text)
        print(f"Recommendation:\n{recommendation}\n")
        print("-" * 50 + "\n")


In [7]:
if __name__ == "__main__":
    main()

🎵 Event Recommender Test 🎵

Fetching events from database...
Found 13 events

--- Test 1: 'I want to see some techno this weekend' ---
Recommendation:
"Hey! I've got some sick techno recs for you this weekend! Here are my top picks:

* SNEAK RAVE on Sep 2nd at XOYO - This one's a no-brainer! A sneak rave is all about underground vibes and techno beats. You won't wanna miss it!
* Steely Dancing on Sep 2nd at Ridley Road Market Bar - This event might not scream techno, but trust me, it's a great chance to dance the night away with some awesome tunes. Plus, it's on a Sunday, so you can recover from the night before 
* PARADOX on Sep 3rd at XOYO - Another XOYO event, another techno rave! PARADOX is likely to feature some of the best techno DJs in London.

Hope you find something that gets you moving!"

--------------------------------------------------

--- Test 2: 'Looking for house music events next week' ---
Recommendation:
"Hey! I've got some sick house music event recs for you in Lond